In [1]:
import os
from langchain_community.graphs import Neo4jGraph

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "2wsx3edc"

database = os.environ['NEO4J_DATABASE']
graph = Neo4jGraph(database=database)

In [2]:
import sys
sys.path.append('..')

from langchain_community.document_loaders import PyPDFLoader

from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, separators=["\n\n", "，", "。", "【", ","])


In [3]:
from tools.graph_builder import TwlfGraphBuilder
graph.query('MATCH (n) DETACH DELETE n;')

graph_builder = TwlfGraphBuilder(graph)

dir = '../data'
for filename in os.listdir(dir):
    docs = []
    if filename.endswith(".pdf"):
        loader = PyPDFLoader(os.path.join(dir, filename))
        doc_pages = loader.load()
        graph_builder.chunkgraph_build_from_one_file(doc_pages, text_splitter)

In [ ]:
# from langchain_openai import AzureOpenAIEmbeddings

# embedding = AzureOpenAIEmbeddings(
#     model="text-embedding-ada-002",
#     azure_endpoint='https://lang-chain-dev.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-05-15',
#     azure_deployment='text-embedding-ada-002',
#     openai_api_version='2023-05-15'
# )

In [4]:
# ! pip3 install -U langchain-huggingface
import os
os.environ['SENTENCE_TRANSFORMERS_HOME'] = '../embedding_model'
from langchain_huggingface import HuggingFaceEmbeddings
# Choose from https://huggingface.co/spaces/mteb/leaderboard
embedding = HuggingFaceEmbeddings(model_name="lier007/xiaobu-embedding-v2")

/Users/andy/miniforge3/envs/lagch/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/andy/miniforge3/envs/lagch/lib/python3.12/site-packages/sentence_transformers/models/Dense.py:89: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explici

In [5]:
from langchain_community.vectorstores import Neo4jVector

graph_store = Neo4jVector.from_existing_graph(embedding=embedding, 
                                    index_name="chunk_index",
                                    node_label='__Chunk__', 
                                    embedding_node_property='embedding', 
                                    text_node_properties=['content'])

In [ ]:
# from langchain_openai import AzureChatOpenAI
# import os
# llm = AzureChatOpenAI(
#     azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
#     azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
#     openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
# )

In [6]:
from langchain_ollama import ChatOllama
llm = ChatOllama(
    model="llama3.1",
)

In [7]:
from langchain.chains import GraphCypherQAChain

graph.refresh_schema()
graph_tradition_chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True, validate_cypher=True)
graph_tradition_tool = graph_tradition_chain.as_tool()
graph_tradition_tool


/Users/andy/miniforge3/envs/lagch/lib/python3.12/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


StructuredTool(name='GraphCypherQAChain', description="Takes {'title': 'ChainInput', 'type': 'object', 'properties': {'query': {'title': 'Query'}}}.", args_schema=<class 'pydantic.v1.main.ChainInput'>, func=<function convert_runnable_to_tool.<locals>.invoke_wrapper at 0x352b8fd80>, coroutine=<function convert_runnable_to_tool.<locals>.ainvoke_wrapper at 0x352b8fc40>)

In [25]:
vectorstore = Neo4jVector.from_existing_graph(embedding=embedding, 
                                    index_name="chunk_index",
                                    node_label='__Chunk__', 
                                    embedding_node_property='embedding', 
                                    text_node_properties=['content'])
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'score_threshold': 0.9}
)
retriever

VectorStoreRetriever(tags=['Neo4jVector', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.neo4j_vector.Neo4jVector object at 0x352dc7b90>, search_type='similarity_score_threshold', search_kwargs={'score_threshold': 0.9, 'k': 3})

In [26]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [27]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    """
    "你是一個有用的助手, 你的任務是回答問題."
    "你必須根據以下提供的檢索內容及資料庫查詢結果進行問答問題."
    "如果檢索內容為空, 則回答 '沒有找到相關資訊'"
    "以 10 話回應, 保持答案的簡潔"
    "以下為檢索內容:\n\n"
    "{context}"

    "以下為資料庫查詢結果:\n\n"
    "{tradition_result}"

    問題: {question}
    """
)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough(), "tradition_result": graph_tradition_chain}
    | prompt
    | llm
    | StrOutputParser()
)

In [28]:
question = "未滿15歲被保險人死亡，身故保險金該怎麼賠?"

In [29]:
retriever.invoke(question)

[Document(metadata={'page_num': 1}, page_content='\ncontent: ，以累計儲存生息之金額一次計算增額繳清保險金額，其後保單年度適用第一項規定。  \n要保人終止本契約，或被保險人於保險年齡達 16歲前死亡者，本公司應退還歷年累計儲存生息之金額予要保人\n。 \n本公司於每一保單年度屆滿後 ，應將該增值回饋分享金之金額，以書面或電子郵件方式通知要保人。  \n \n【身故保險金或喪葬費用保險金的給付】  \n被保險人於本契約有效期間內身故者，本公司按下列三款取其最大值給付身故保險金：  \n一、身故日之當年度保險金額。  \n二、身故日之保單價值準備金 乘以保單價值準備金比率所得之金額。  \n三、身故日之年繳應繳保險費總和的 1.06倍。  \n訂立本契約時，以未滿 15足歲之未成年人為被保險人，除喪葬費用之給付外，其餘死亡給付之約定於被保險人滿\n15足歲之日起發生效力；被保險人滿 15足歲前死亡者，其身故保險金變更為喪葬費用保險金。  \n前項未滿 15足歲之被保險人如有於民國九十九年二月三日 (不含 )前訂立之保險契約，其喪葬費用保險金之給付依下\n列方式辦理：  \n一、被保險人於民國九十九年二月三日 (不含 )前訂立之保險契約，喪葬費用保險金額大於或等於遺產及贈與稅法第\n十七條有關遺產稅喪葬費扣除額之半數 (含)者，其喪葬費用保險金之給付，從其約定，一百零九年六月十二日 (\n含)以後所投保之喪葬費用保險金額，本公司不負給付責任，並應無息退還該超過部分之已繳保險費。  \n二、被保險人於民國九十九年二月三日 (不含 )前訂立之保險契約，喪葬費用保險金額小於遺產及贈與稅法第十七條'),
 Document(metadata={'page_num': 2}, page_content='\ncontent: 第 2 頁，共  4 頁 一、被保險人於民國九十九年二月三日 (不含 )前訂立之保險契約，喪葬費用保險金額大於或等於\n遺產及贈與稅法第十七條有關遺產稅喪葬費扣除額之半數 (含)者，其喪葬費用保險金之給付\n，從其約定，一百零九年六月十二日 (含)以後所投保之喪葬費用保險金額，本公司不負給付\n責任，並應無息退還該超過部分之已繳保險費。  \n二、被保險人於民國九十九年二月三日 (不含 )前訂立之保

In [13]:
graph_tradition_chain.invoke(question)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:Chunk)-[:NEXT]->(n) RETURN c, n LIMIT 10
Full Context:
[{'c': {'page_num': 1, 'embedding': [-0.5891573429107666, -1.289288878440857, -1.2155100107192993, -0.38512495160102844, 1.501043677330017, 1.5137169361114502, 0.06288518011569977, 0.5458245873451233, -0.11910190433263779, -0.4284265637397766, 0.6274601817131042, 0.22760431468486786, -0.3344429135322571, -0.08254880458116531, 0.8936185240745544, 1.3188539743423462, -1.4397574663162231, 0.5922224521636963, -0.9558241367340088, 0.20096349716186523, 0.4309820532798767, -0.6693037152290344, 0.031566914170980453, 0.10670401901006699, 0.7652047872543335, 0.43483778834342957, -0.3473667502403259, -1.0897263288497925, -0.8639450073242188, 0.9672914743423462, 0.25272563099861145, -0.1539900004863739, 0.13677525520324707, -1.3909016847610474, 0.35739633440971375, 0.6703830361366272, -0.00044850842095911503, -0.1941249668598175, 0.9504159092903137, 0.857210636138916, -0.7

{'query': '未滿15歲被保險人死亡，身故保險金該怎麼賠?',
 'result': '根據提供的「除外責任」條款，未滿15歲被保險人的死亡不屬於自殺或故意致死，因此本公司仍負給付身故保險金之責任。\n\n所以，如果未滿15歲的被保險人死亡，保險公司應該賠付身故保險金。'}

In [18]:
rag_chain.invoke(question)

'根據第 2 頁的文內容，如果未滿 15 歲的被保險人於民國一百零九年六月十二日（含）以後及第四項被保險人的投保情形，喪葬費用保險金應該按照以下條件計算：\n\n1. 單位數量：不得超過遺產稅喪葬費扣除額之半數。\n2. 超過部分：本公司不負給付責任，並應無息退還該超過部分之已繳保險費。\n\n所以，如果未滿 15 歲的被保險人死亡，身故保險金將按照以上條件進行計算。'